In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

import os
os.chdir('/home/cat/code/manifolds/')

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange, tqdm

from scipy.io import loadmat

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.spatial import ConvexHull, convex_hull_plot_2d

#from ensembles.utils import load_UMAP, load_binarized_traces, find_ensemble_order, load_data, HMM, get_footprint_contour, load_footprints, computing_ensemble_loadings




Autosaving every 180 seconds


In [19]:
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'
session = '20210520'

binarization_method='upphase'

#
c = calcium.Calcium()
c.root_dir = root_dir
c.animal_id = animal_id
c.session = session
c.load_suite2p()

# 
c.load_binarization()
if binarization_method=='onphase':
    traces = c.F_onphase_bin
elif binarization_method=='upphase':
    traces = c.F_upphase_bin
else:
    print ("METHOD NOT FOUND")


print ("binarized data: ", traces.shape)

w = wheel.Wheel()
w.root_dir = os.path.join(root_dir,
                          animal_id,
                          session,
                          'TRD-2P')
w.animal_id = animal_id
w.session = session
w.load_track()
w.compute_velocity()




(584, 55740)
binarized data:  (584, 55740)


In [6]:
from gini import *
spars = []
binwidth = 1
rasters = traces.T
for k in trange(0, rasters.shape[0], binwidth):
    temp = rasters[k:k+binwidth].sum(0)
    #print (temp.shape)
    spars.append(gini(temp))
    



100%|██████████████████████████████████| 55740/55740 [00:04<00:00, 11725.70it/s]


In [14]:
#
spars =np.array(spars)
idx =np.where(spars<0.8)[0]
print (idx.shape[0]/spars.shape[0])
y = np.histogram(spars,bins=np.arange(-0.1,1.1
                                      ,0.01))

fig=plt.figure()
plt.bar(y[1][:-1],
        y[0],
       0.01/2)

plt.title("1.0 represents highly sparse data; 0.0 represents highly homogenous data")
plt.xlabel("Sparsity index")
plt.ylabel("# of frames")
#plt.semilogy()
#plt.ylim(bottom=0.1)
plt.show()

0.1653031933979189


In [27]:
idx1 = np.where(spars>0.5)[0]
idx2 = np.where(spars<=0.5)[0]
print (idx1.shape)

plt.plot(w.track.velocity.times, 
        w.track.velocity.values, 
         c='red',
         linewidth=3,
         label='velocity')

plt.plot(w.track.velocity.times,
         c.F_upphase_bin.sum(0)/100., 
         c='green',
         linewidth=3,
         label='mua')

plt.scatter(w.track.velocity.times[idx1], 
            np.zeros(idx1.shape[0]), 
            s=200,
            c='black', label='high sparsity')
plt.scatter(w.track.velocity.times[idx2],
            np.zeros(idx2.shape[0]), 
            s=200,
            c='blue',
            label='low sparsity')




plt.legend()
plt.show()


(46526,)
